# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>

## TD3 : Régression logistique pour prédire la survie des passagers du Titanic

Pour ce concours, vous aurez accès aux caractéristiques des passagers du Titanic

La liste des passagers a été scindé en deux groupes : un groupe d'entraînement (*Train.csv*) et un groupe de test (*Test.csv*). Les membres d'équipages sont ignorés. Le jeu de données d'entraînement doit être utilisé pour estimer les paramètres d'un modèle de régression logistique. Vous devrez trouver le meilleur modèle possible en faisant un choix judicieux des variables explicatives. Par la suite, votre modèle servira à prédire le sort des passagers de la liste de test.

L'ensemble d'entraînement contient les informations de 872 passagers et leur sort (décès ou survie).

L'ensemble de test contient seulement les informations de 437 passagers. Vous devez prédire pour chacun d'eux s'ils ont survécu ou non au naufrage.

Avant d'exécuter ce code, vous devrez récupérer les informations sur les passagers du Titatnic déjà partitionnées en échantillon d'apprentissage et de test. Vous modèles pourront être entraînés sur les données *Train.csv* disponible sur la page GitHub du cours. Vos prédictions seront par la suite comparées au sort des passager de l'échantillon de test. Vous devrez déposer voter fichier de prédictions sur la [page Kaggle du TD3](https://www.kaggle.com/t/35b16b5d9e4149808d2a155b93fecd21).



Dans ce code de base, on n'utilisera que la classe du passager pour prédire sont sort. **L'objectif de ce TD est d'améliorer le taux de bonnes prédictions en ajoutant des variables explicatives significatives.**


### Sommaire:
___

1. **[Chargement des données de l'ensemble d'entraînement](#section_1)**   
2. **[Analyse exploratoire](#section_2)**  
3. **[Modèle de régression logistique](#section_3)**
    1. Encodage de la variable classe à l'aide de deux variables indicatrices
    2. Estimation des paramètres du modèle de régression logistique   
4. **[Prédiction de la survie des passagers de l'ensemble de test](#section_4)**
    1. Chargement des données de l'ensemble de test
    2. Encoder la classe des passagers de l'ensemble de test avec deux variables indicatrices
    3. Estimation de la probabilité de survie de chacun des passagers de l'ensemble de test
    4. Prédiction de la survie des passagers de l'ensemble de test
    5. Préparation du fichier des préditions pour téléverser sur Kaggle


In [ ]:
# Chargement des librairies nécessaires

using CSV, DataFrames, Gadfly, GLM, Distributions, LinearAlgebra

<a id="section_1"></a>
### 1. Chargement et traitement des données


In [ ]:
data = CSV.read("Train.csv")
first(data,5)

<a id="section_2"></a>
### 2. Analyse exploratoire

On souhaite vérifier s'il existe une relation entre la classe des passagers et leur taux de survie.

In [ ]:
df = by(data, :Pclass, df -> mean(df[:,:Survived]))
rename!(df, :Pclass => :Classe, :x1 => :Taux_Survie)
plot(df, x=:Classe, y=:Taux_Survie, Geom.point, Geom.hair)

<a id="section_3"></a>
### 3. Modèle de régression logistique 

La classe des passagers est utilisée comme variable explicative de la probabilité de survie.

#### 3.1 Encodage de la variable classe à l'aide de deux variables indicatrices

In [ ]:
# Deux variables indicatrices pour la classe des passagers

Pclass = data[:,:Pclass]

n = length(Pclass)
x₁ = zeros(n)
x₂ = zeros(n)

for i = 1:n
    if Pclass[i]==2
        x₂[i] = 1
    elseif Pclass[i]==1
        x₁[i] = 1
    end   
end

#### 3.2 Estimation des paramètres du modèle de régression logistique

Le lien logit est utilisé.

In [ ]:
# On stocke les données dans un dataframe pour utiliser la package GLM de Julia
df = DataFrame(x₁=x₁,x₂=x₂,Y=data[:,:Survived])

M = glm(@formula(Y ~ x₁+x₂), df,  Bernoulli(), LogitLink())

<a id="section_4"></a>
### 4. Prédiction de la survie des passagers de l'ensemble de test

#### 4.1 Chargement des données de l'ensemble de test

In [ ]:
# Charger le jeu de données de test
test = CSV.read("Test.csv");

#### 4.2 Encoder la classe des passagers de l'ensemble de test avec deux variables indicatrices

In [ ]:
# Encoder la classe de chacun des passagers de l'échantillon de test

Pclass = test[:,:Pclass]
n = length(Pclass)
x̃₁ = zeros(n)
x̃₂ = zeros(n)
for i = 1:n
    if Pclass[i]==2
        x̃₂[i] = 1.0
    elseif Pclass[i]==1
        x̃₁[i] = 1.0
    end   
end

X̃ = hcat(ones(Int64,n),x̃₁,x̃₂)

#### 4.3 Estimation de la probabilité de survie de chacun des passagers de l'ensemble de test

In [ ]:
θ̂ = predict(M, X̃)

#### 4.4 Prédiction de la survie des passagers de l'ensemble de test

In [ ]:
# Prédiction de la survie pour chacun des passagers de l'échantillon de test

Ŷ = zeros(Int64,n)
Ŷ[θ̂.>.5] .= 1;

#### 4.5 Préparation du fichier des préditions pour téléverser sur Kaggle

In [ ]:
# Enregistrement dans un fichier CSV pour soumission sur Kaggle
# Première colonne : identifiant des passagers
# Deuxième colonne : survie = 1, décès = 0 (nombres entiers)

prediction = DataFrame(PassengerId = test[:,:PassengerId], Survived = Ŷ)
CSV.write("benchmark_predictions.csv",prediction)